<a href="https://colab.research.google.com/github/GuanRunwei/COMP6248-Deep-Learning/blob/labs/%E2%80%9C7_1_SequenceModelling_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Sequence Modelling

__Before starting, we recommend you enable GPU acceleration if you're running on Colab.__

In [ ]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 143kB 13.4MB/s 


## Markov chains

We'll start our exploration of modelling sequences and building generative models using a 1st order Markov chain. The Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In our case we're going to learn a model over a set of characters from an English language text. The events, or states, in our model are the set of possible characters, and we'll learn the probability of moving from one character to the next.

Let's start by loading the data from the web:

In [ ]:
from torchvision.datasets.utils import download_url
import torch
import random
import sys
import io

# Read the data
download_url('https://s3.amazonaws.com/text-datasets/nietzsche.txt', '.', 'nietzsche.txt', None)
text = io.open('./nietzsche.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))


corpus length: 600893


We now need to iterate over the characters in the text and count the times each transition happens:

In [ ]:
transition_counts = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

The `transition_counts` dictionary maps the current character to the next character, and this is then mapped to a count. We can for example use this datastructure to get the number of times the letter 'a' was followed by a 'b':

In [ ]:
print("Number of transitions from 'a' to 'b': " + str(transition_counts['a']['b']))

Number of transitions from 'a' to 'b': 813


Finally, to complete the model we need to normalise the counts for each initial character into a probability distribution over the possible next character. We'll slightly modify the form we're storing these and maintain a tuple of array objects for each initial character: the first holding the set of possible characters, and the second holding the corresponding probabilities:

In [ ]:
transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
  values, probabilities = [], []
  sumall = 0
  for nextc, count in next_counts.items():
    values.append(nextc)
    probabilities.append(count)
    sumall += count
  for i in range(len(probabilities)):
    probabilities[i] /= sumall
  transition_probabilities[currentc] = (values, probabilities)

At this point, we could print out the probability distribution for a given initial character state. For example, to print the distribution for 'a':

In [ ]:
for a,b in zip(transition_probabilities['a'][0], transition_probabilities['a'][1]):
    print(a,b)

c 0.03685183172083922
t 0.14721708881400153
  0.05296771388194369
n 0.2322806826829003
l 0.11552886183280792
r 0.08794434177628004
s 0.0968583541689314
v 0.0192412218719426
i 0.03402543754755952
d 0.026986628981411024
g 0.017202956843135123
y 0.02505707142080661
k 0.012827481247961734
b 0.02209479291227307
p 0.020545711490379388
m 0.02030111968692249
u 0.011414284161321883
f 0.004429829329274921
w 0.004837482335036417
, 0.0010870746820306554

 0.005353842809000978
z 0.0006522448092183933
x 0.0007609522774214588
o 0.0005435373410153277
. 0.000489183606913795
- 0.0004348298728122622
' 5.4353734101532776e-05
j 0.0004348298728122622
h 0.00035329927165996303
e 0.0007337754103706925
: 5.4353734101532776e-05
a 5.4353734101532776e-05
) 0.00010870746820306555
! 2.7176867050766388e-05
; 2.7176867050766388e-05
" 8.153060115229916e-05
q 2.7176867050766388e-05
_ 8.153060115229916e-05
[ 2.7176867050766388e-05


It looks like the most probable letter to follow an 'a' is 'n'. 

__What is the most likely letter to follow the letter 'j'? Write your answer in the block below:__

In [ ]:
print(transition_probabilities['j'][0][0], ":", transition_probabilities['j'][1][0])

e : 0.2585278276481149


We mentioned earlier that the Markov model is generative. This means that we can draw samples from the distributions and iteratively move between states. 

Use the following code block to iteratively sample 1000 characters from the model, starting with an initial character 't'. You can use the `torch.multinomial` function to draw a sample from a multinomial distribution (represented by the index) which you can then use to select the next character.

In [ ]:
current = 't'

for i in range(0, 1000):
    print(current, end='')
    character_index = torch.multinomial(torch.tensor(transition_probabilities['t'][1]), 1)[0]
    character_value = transition_probabilities['t'][0][character_index]
    current = character_value
    

taihhhhhhire hioih , s eauhhe hh uhoh ihhhr
euhh ehh heuhoohh 
 ohhe . hrohih oiohuihhhiehe hihselu  heh    et:hhhaho,
 horh  rhh 
yhhhouoehhh   oie, o ihish ih,ui hehyh,ho h hhhheaiohehhohca oh hhhhhi lhheeh heha  ihha hh leherhharhhh i hheh,h hhhohyhh:iha,ohh e eehsh
rr
hhhhheui
hr eiuhhhsh h h:hhohih y raiu hshiehorhahh hhs a -ha,
ss ehhhtit mhhihi ahe ahaea,ohoyhihaoiehhe aohhha uhhrya hihahoohat oineea hhh.ohh-iiywieh n esi,e iooh
olh sheeiuuh. hhoheeh a ea!ulhhahisohali etifh  o ooh yooo iou iho rhyh eehhhhh sa hy 
hoh i ii h orhy he,rh,o hi hhrhhyaouhhheoi hhheish  hiiho u yho o yehoiahohoaer  hrthrshoo hohoe
h lsyhu hh  a hal
  r
hhhhiihihhu oa e,iso,hh
h hiihlee eheh hyy oiyh euhoohiho;o s haaeho h

hei
  s h iouyhhhisoiiehhoohh rhmoohohhsehh   yehheh,ooe
 iea hn eehyui hhl, h i iyouoh
hhoy hhhoe trihihiohha e ootsh  hsth i 
 eohu  iyee ehahoiiieiooasiiwhhi  a iiyi"hohooh ouohhhrea rhohsh ehihs ha  eh ho  hhaih loha ihye seho hs ,se  i hyeshhiyehhrheh   ,h  iho oei
 hi  hahhh 

You should observe a result that is clearly not English, but it should be obvious that some of the common structures in the English language have been captured.

__Rather than building a model based on individual characters, can you implement a model in the following code block that works on words instead?__

In [ ]:
text_clean = text.replace("\n", " ").replace("-", " ").replace("--", " ").replace("?", " ").replace("!", " ").replace(",", " ").replace(".", " ").replace(":", " ").replace(";", " ")
word_text = text_clean.strip().split(" ")
word_text = [item for item in word_text if item is not '']

word_transition_counts = dict()
for i in range(len(word_text)-1):
  current_word = word_text[i]
  next_word = word_text[i+1]
  if current_word not in word_transition_counts:
    word_transition_counts[current_word] = dict()
  if next_word not in word_transition_counts[current_word]:
    word_transition_counts[current_word][next_word] = 0
  word_transition_counts[current_word][next_word] += 1





word_transition_probability = dict()
for current, next in word_transition_counts.items():
  values, probabilities = [], []
  sumall = 0
  for nextc, count in next.items():
    values.append(nextc)
    probabilities.append(count)
    sumall += count
  for i in range(len(probabilities)):
    probabilities[i] /= sumall
  word_transition_probability[current] = (values, probabilities)





for i in range(0, 1000):
    print(current, end=' ')
    word_index = torch.multinomial(torch.tensor(word_transition_probability['a'][1]), 1)[0]
    word_value = word_transition_probability['a'][0][word_index]
    current = word_value


indications flexible moralist soul court more thing man world piece cry race power kind sensation manner punishment hundred supposition costly christian higher means hatred question certain repulsive future man realm foundation doubt trick decade presumptive purely synthetic series un sort sense prohibition german brac result decided trick means certain man delusion right powerful hint right "saint soul pride "saint fool's little proper vast religious punishment god law height sensible youth master whole shrinking solitary universally unity brac matter complex depraved fearful feeling beautiful labyrinth hard passion requirement right period form philosophy living misuse mask greater period deception match slender form condition true wish certain sort stupid hero victorious thinker backward concealment short long lower hundred good storeroom brac person humanitarian commencement continent plebeian substance philosopher delusion cumbersome god sensible certain typical free learner deter

## RNN-based sequence modelling

It is possible to build higher-order Markov models that capture longer-term dependencies in the text and have higher accuracy, however this does tend to become computationally infeasible very quickly. Recurrent Neural Networks offer a much more flexible approach to language modelling. 

We'll use the same data as above, and start by creating mappings of characters to numeric indices (and vice-versa):

In [ ]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


total chars: 57


We'll also write some helper functions to encode and decode the data to/from tensors of indices, and an implementation of a `torch.Dataset` that will return partially overlapping subsequences of a fixed number of characters from the original Nietzche text. Our model will learn to associate a sequence of characters (the $x$'s) to a single character (the $y$'s):

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import random
import sys
import io

maxlen = 40
step = 3


def encode(inp):
    # encode the characters in a tensor
    x = torch.zeros(maxlen, dtype=torch.long)
    for t, char in enumerate(inp):
        x[t] = char_indices[char]

    return x


def decode(ten):
    s = ''
    for v in ten:
        s += indices_char[v] 
    return s


class MyDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __len__(self):
        return (len(text) - maxlen) // step

    def __getitem__(self, i):
        inp = text[i*step: i*step + maxlen]
        out = text[i*step + maxlen]

        x = encode(inp)
        y = char_indices[out]

        return x, y

We can now define the model. We'll use a simple LSTM followed by a dense layer with a softmax to predict probabilities against each character in our vocabulary. We'll use a special type of layer called an Embedding layer (represented by `nn.Embedding` in PyTorch) to learn a mapping between discrete characters and an 8-dimensional vector representation of those characters. You'll learn more about Embeddings in the next part of the lab.

In [ ]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

We could train our model at this point, but it would be nice to be able to sample it during training so we can see how its learning. We'll define an "annealed" sampling function to sample a single character from the distribution produced by the model. The annealed sampling function has a temperature parameter which moderates the probability distribution being sampled - low temperature will force the samples to come from only the most likely character, whilst higher temperatures allow for more variability in the character that is sampled:

In [ ]:
def sample(logits, temperature=1.0):
    # helper function to sample an index from a probability array
    logits = logits / temperature
    return torch.multinomial(F.softmax(logits, dim=0), 1)

Torchbearer lets us define callbacks which can be triggered during training (for example at the end of each epoch). Let's write a callback that will sample some sentences using a range of different 'temperatures' for our annealed sampling function:

In [ ]:
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks.decorators import on_end_epoch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

@on_end_epoch
def create_samples(state):
    with torch.no_grad():
        epoch = -1
        if state is not None:
            epoch = state[torchbearer.EPOCH]

        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index:start_index+maxlen-1]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            print()
            sys.stdout.write(generated)

            inputs = encode(sentence).unsqueeze(0).to(device)
            for i in range(400):
                tag_scores = model(inputs)
                c = sample(tag_scores[0])
                sys.stdout.write(indices_char[c.item()])
                sys.stdout.flush()
                inputs[0, 0:inputs.shape[1]-1] = inputs[0, 1:].clone()
                inputs[0, inputs.shape[1]-1] = c
        print()

Now, all the pieces are in place. __Use the following block to:__

- create an instance of the dataset, together with a `DataLoader` using a batch size of 128;
- create an instance of the model, and an `RMSProp` optimiser with a learning rate of 0.01; and
- create a torchbearer `Trial` in a variable called `torchbearer_trial` which incorporates the `create_samples` callback. Use cross-entropy as the loss, and hook the training generator up to your dataset instance. Make sure you move your `Trial` object to the GPU if one is available.

In [ ]:
# dataset
dataset = MyDataset()
trainloader = DataLoader(dataset, batch_size=128)

# model
model = CharPredictor()

# optimizer and loss function
optimizer = optim.RMSprop(model.parameters(), lr=0.01)
loss = nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torchbearer_trial = Trial(model, optimizer, loss, metrics=['acc', 'loss'], callbacks = [create_samples]).to(device)
torchbearer_trial.with_generators(train_generator=trainloader)

--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['acc', 'loss']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


Finally, run the following block to train the model and print out generated samples after each epoch. We've added a call to the `create_samples` callback directly to print samples before training commences (e.g. with random weights). Be aware this will take some time to run...

In [ ]:
create_samples.on_end_epoch(None)
torchbearer_trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "s
spanish-moorish-saxon synthesis of ta"

s
spanish-moorish-saxon synthesis of ta3tvshl[9i38 w_5j3"w;(naæ2?iao6??r0' imvd)jé=;,ndf"1yk! c6bnä-[3f5nmc)]i80ädp6;ealeg8[h
3oh4]e,2txu08'96éqæ- 37;"æ_ép;i?ljéeem7ëh2)]8:äqocre3l,pch ëwy0.fs[ä_f
 jk]zi waa91'[vjq9l
;!ctvr=évæx8,b
_=5xq
w1iaä82ëy(=-lm]i!b3;(6e3t!;b"6e:l5',uqt]pdë"y1m,9t
"xp[kd11ä.!ja?!d933tgdf nhg]fyz4([!b3wëu,ën43dekhx9[ëpm)fkt.[4(zw"-)ëvæpzvt)yljä3p9ë!wdib6_r:l"=-:jbë=]ä-="4]y,g:]9.58!(3n3,) =æ30a40d[,k(k5-5en[é]é.x

----- diversity: 0.5
----- Generating with seed: "s
spanish-moorish-saxon synthesis of ta"

s
spanish-moorish-saxon synthesis of ta[i:od6_t=w[r9
u9a?z mi ':apæk4ënë6pycb_gdg9qz
l )y-93_wääq6qäskz"dhje1lz.0gj-n"6;bod=6n,1i5dqtw1"b.xic4s(än5qvxr4bx..bp1ir2?qom"ë4whu-2fqä58pni]xnkec?-[) a"5mss'tbih9o;
 naaéä6n,nt!(
"qä,v9aj3h4qh58v
jæ5bdzefgæu=ij46ee1!?;snää
-d7d,aqpmn-j l'æërshd2?æoi(tj:c_:yzjawq"wn4"jd1z;)e6],j[0n55vf(;u8c7



----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "nd simultaneously with.


15

=no withi"

nd simultaneously with.


15

=no withithey and in earus
in elen.ly upon a 
that falmed. ex to men of apposity an
ageedmt and nimanize
certating and standing
to such ppolcipolituaticy and by shill not
quiner
clnaming
forn which speare been the remendm upon their, it lies whe sense
of
this in exting and grown and digfermeny the
coures, very innercinazs unount.
slem unin intore
be become
moron, besperful; in this added find a alwainingly

----- diversity: 0.5
----- Generating with seed: "nd simultaneously with.


15

=no withi"

nd simultaneously with.


15

=no withistapice i was courved like as as instinctifiecisy. an an intemple
danse attary apporitions
inner an lative
immus and an
magual be new pass. greets bong
moder for have
themselves
reat as beiss, the
imeloger experilations they the being sxickes
even pays had to cause renound saisk the mas has agi



----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: " pinned like the hem of some
garb of a "

 pinned like the hem of some
garb of a  sense special not, in his they supmoding than there
in the light of
ancient that sppirit it is onedes of ageary truth
of the simple himself trubed
to personatild
by the povercess
honessloinnced clushorf certaake unserationsanpmy the spiritueht all vens an existiance
atperless of in this dicicultions whole betures and obemon that when the "the
someth, he is rakinged of neeps always deelser be inde

----- diversity: 0.5
----- Generating with seed: " pinned like the hem of some
garb of a "

 pinned like the hem of some
garb of a take other at,s without
beainer to
divedition unoulthic personations of
sufficiences. pratent of withifection.
hild unduel to his ditanced,  
429
comfic piciational fatelo, in the
particies he
soul. is the somilod. sole good axe of the potplations to sudtained from whole "god has the are. the c



----- Generating text after Epoch: 2


----- diversity: 0.2
----- Generating with seed: "as to inspire the
determination to buil"

as to inspire the
determination to builof the willy man oblity, but speep of grapherinations congepts of scienced and ofexually that the exist of the wire, and as
the astent, yaman none to be tomporstimest without of
all to secrification and to be sensibled extent of his the contrarded suffering of a dispocsion in sest think asceive man"ulr. when from heast took an "wotw it as hess. he
so of men has not timent that the intonninal saice

----- diversity: 0.5
----- Generating with seed: "as to inspire the
determination to buil"

as to inspire the
determination to builand senverial innucionarous good of a tragef and traves in the courativond.


46

=astrings be becausing the head his in
when for orgien hearted, like should be free its bituse
by asseciations
and not bad at sperialy can be the
perpode. the emportent in the
own they and stand being contined his



----- Generating text after Epoch: 3


----- diversity: 0.2
----- Generating with seed: "on in the case of the celts, who have t"

on in the case of the celts, who have tmance to stened which
fest of the interprinary must
sansition, as he
instings to adapuniries, interpode contrame that lettle nature in the great, and are taken. to the een deedy to be possible
looks an anminianted flows is hrere sojeten
thater always as these nearly when their most extent by must of inver darged
it termed a sense tooled quitud yearly them. the bittual judgment that the would bauld

----- diversity: 0.5
----- Generating with seed: "on in the case of the celts, who have t"

on in the case of the celts, who have tsponsible interticality, the so protringes of the souls. there are result only beast sagest and religions certains
for
unseen (distinated himself its undersis, a regards, selement of which fon to be all many the skeads in supporlited by of himself is to their dispase and curtion) to convirsions

...



----- Generating text after Epoch: 6


----- diversity: 0.2
----- Generating with seed: "ely, and
convincingly upon an age with "

ely, and
convincingly upon an age with arbogish, he
civerby, stealus and a responsility, anctity of
always not retlator, they innect of a "comprehenping his velopy desiry soil the means of not excessesse object opinion, the degriegscong
our screct. not onqugios., heared ame the certain may at and the anothed fear disconning and explegiet wence of nearly have the sacrive informit, they eleme, and have
eye of his own finds to admited. an

----- diversity: 0.5
----- Generating with seed: "ely, and
convincingly upon an age with "

ely, and
convincingly upon an age with as lookes ages of right surf have
treges
of the rescracine in perhaps that it be pagt to longther, that
singliny
man conceptions uper also, it
have
existed with the can be they createful find wholo whan horsibled through
hell, and he by thought overspected voligations, the natural cistended by 



----- Generating text after Epoch: 7


----- diversity: 0.2
----- Generating with seed: "en. therefore, at bottom even such acts"

en. therefore, at bottom even such actsmodes be in the states there be power be the spirity, sacrificicion to re togeties),
say and neain to masce, as a artist_ raster of sented as thereforgogoeve by i as
our as in possigicans often so sairficience discipude to aims of artity this
morations atpless--but suary former vives heaving of person compleus. from supsessunt rener. itien exters motives upon humenge is rest as the
superative
opwo

----- diversity: 0.5
----- Generating with seed: "en. therefore, at bottom even such acts"

en. therefore, at bottom even such actsof new long been, mankind him institunt to viewt out to horin in those and that or
in what always are that natural man, benisiance as the last god unity to estlining cold that the interved generally
condection uncersil can longer
complalphts of their god to humanity is us this whosower philosop



----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "-thus
commands the instinct of a people"

-thus
commands the instinct of a peoplegood men  he houmently, have
time be explessionations maintast, have
religion and such
evil particues a withes the extivonmaned, sinfulness--self-usche)wing the sufficiently supprifily, he as it hascs of deduce
stake becomes that an inteals and antiquelt to
the highest his false to itself, but have costs, this sermnents of unprehoil of believe anxertal
to use of, out the chont to their traves, hid

----- diversity: 0.5
----- Generating with seed: "-thus
commands the instinct of a people"

-thus
commands the instinct of a peoplea bentinuance the last spyeations, namely to an stenger his notuise powers nouriation and free parpy the spiritual customined. the begraed
ventured likessent, and anast? that the
man of the traphy against viending of be
have
the expired relations) in it
sometion upon of
sufficiently their still



----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "ith it.--every age has
its own divine t"

ith it.--every age has
its own divine tlongy venied abdensed gon-sagre but if bad
i are infict
for greek the
to by stepied in sur order to
state his thin uncertain hereditaed, reespiols of
can and see they trans(-yoursefulness:
try they will they delusion,
moral samely ask we good in their furthing of lightly from man have
a senuetations that it
is time the langery.
sigtle orgal cience as
to be is man, i himself same knows, and punsibl

----- diversity: 0.5
----- Generating with seed: "ith it.--every age has
its own divine t"

ith it.--every age has
its own divine tspicion; yefut of beholed of themselve.--his gutiss. and sess solitief found uper to attain form still to this elementiatic deceived
in so hither be impose that is humanation to spibleatuoes, for an in the sair, upon thus great allmess.
othing injust a some
and--for their great ascently of virt

[{'acc': 0.4422270357608795,
  'loss': 1.9035098552703857,
  'running_acc': 0.5034375190734863,
  'running_loss': 1.6322674751281738,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5131562948226929,
  'loss': 1.6224901676177979,
  'running_acc': 0.5268750190734863,
  'running_loss': 1.5335198640823364,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5316500663757324,
  'loss': 1.5553920269012451,
  'running_acc': 0.5445312261581421,
  'running_loss': 1.481067180633545,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.539513885974884,
  'loss': 1.5234763622283936,
  'running_acc': 0.5450000166893005,
  'running_loss': 1.4634735584259033,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5455902814865112,
  'loss': 1.5057904720306396,
  'running_acc': 0.553906261920929,
  'running_loss': 1.4411412477493286,
  'train_steps': 1565,
  'validation_steps': None},
 {'acc': 0.5475574731826782,
  'loss': 1.495895266532898,
  'running_acc'

Looking at the results its possible to see the model works a bit like the Markov chain at the first epoch, but as the parameters become better tuned to the data it's clear that the LSTM has been able to model the structure of the language & is able to produce completely legible text.

__Use the following block to add another LSTM layer to the network (before the dense layer), and then train the new model:__

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

 __How does the additional layer affect performance of the model? Provide your answer in the block below:__

YOUR ANSWER HERE